In [ ]:
import  pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import datetime
import re

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_train=pd.read_csv("train.csv",index_col="UniqueID")
df_test=pd.read_csv("test.csv",index_col="UniqueID")
df_test["loan_default"]=-999
df=pd.concat([df_train,df_test],axis=0,sort=False)

In [ ]:
df_test_new=pd.read_csv("test.csv",index_col="UniqueID")

In [ ]:
def get_date(date):
    try:
        date.index("-",0)
        if (int(date[6:8])>=19):
            date = date[3:5]+"/"+date[0:2]+"/19"+date[6:8]
        else:
            date = date[3:5]+"/"+date[0:2]+"/20"+date[6:8]
        return datetime.datetime.strptime(date,"%m/%d/%Y").strftime('%m/%d/%Y')
    except ValueError:
        date = datetime.datetime.strptime(date,"%m/%d/%Y").strftime('%m/%d/%Y')
    return date

vget_date = np.vectorize(get_date)

df["Date.of.Birth"]=vget_date(df["Date.of.Birth"])
df["DisbursalDate"]=vget_date(df["DisbursalDate"])

In [ ]:
def get_date_diff(date1,date2):
    date1_out = datetime.datetime.strptime(date1,"%m/%d/%Y")
    date2_out = datetime.datetime.strptime(date2,"%m/%d/%Y")
    return (date1_out-date2_out).days/365

vget_date_diff=np.vectorize(get_date_diff)

df["Age"]=vget_date_diff(df["DisbursalDate"],df["Date.of.Birth"])

In [ ]:
#replace missing values :Employment.Type
df['Employment.Type'].fillna('Not employed',inplace=True)
# create new column
df['Employment.Type'] = df["Employment.Type"].map({'Not employed':0,'Self employed':1, 'Salaried':2})

In [ ]:
#AVERAGE.ACCT.AGE
def get_month(str_t):
    try:
        yr_ind = str_t.index("yrs",0)
        mo_ind = str_t.index("mon",0)
        b_ind = str_t.index(" ",0)
        
        mon = int(str_t[0:yr_ind])*12+int(str_t[b_ind+1:mo_ind])
    except:
        mon = 0
    
    return mon/12

vget_month = np.vectorize(get_month)
df["AVERAGE.ACCT.AGE"] = vget_month(df["AVERAGE.ACCT.AGE"])

In [ ]:
#CREDIT.HISTORY.LENGTH
df["CREDIT.HISTORY.LENGTH"] = vget_month(df["CREDIT.HISTORY.LENGTH"])

In [ ]:
#"Date.of.Birth","DisbursalDate"
df.drop(["Date.of.Birth","DisbursalDate","MobileNo_Avl_Flag","disbursed_amount","asset_cost","CREDIT.HISTORY.LENGTH","PRI.NO.OF.ACCTS","PRI.ACTIVE.ACCTS","SEC.SANCTIONED.AMOUNT","PRI.SANCTIONED.AMOUNT","PERFORM_CNS.SCORE.DESCRIPTION","SEC.NO.OF.ACCTS","VoterID_flag"],axis=1,inplace=True)

In [ ]:
tmp = df["loan_default"]
df.drop(["loan_default"],axis=1,inplace=True)

In [ ]:
df = pd.concat([df,tmp],axis=1)

In [ ]:
df.info()#branch_id,manufacturer_id,State_ID

In [ ]:
#one hot encoding of geography column
from sklearn.preprocessing import OneHotEncoder
onehotencode=OneHotEncoder(categorical_features=[1,3,6])
df_new=onehotencode.fit_transform(df)

In [ ]:
type(df_new)

In [ ]:
df=pd.DataFrame(df_new.todense())

In [ ]:
df.head()#117,120,118

In [ ]:
df_train = df[df[140]!=-999]

In [ ]:
df_test = df[df[140]==-999]

In [ ]:
X = df_train.iloc[:,0:-1]
y = df_train.iloc[:,-1]
df.info()

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=0)

In [ ]:
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [ ]:
categorical_features=[117,118,120]

In [ ]:
parameters={'reg_alpha':[0.1,0.5],
           'subsample':[0.8,0.9],
           'num_leaves':[50,64],
           'subsample_for_bin':[10000,20000]}

In [ ]:
lr=LGBMClassifier(n_estimators=10000, objective = 'binary',
                                   class_weight = 'balanced', learning_rate = 0.001,
                                 reg_lambda = 0.1, 
                                   random_state = 50)


In [ ]:
grid_obj = GridSearchCV(lr, parameters)

In [ ]:
grid_obj = grid_obj.fit(X_train, y_train)

In [ ]:
clf = grid_obj.best_estimator_

In [ ]:
# Fit the best algorithm to the data. 
clf.fit(X_train, y_train)

In [ ]:
y_predict=clf.predict(X_test)
print(y_predict[0:20])
#vround=np.vectorize(round)
#y_predict=vround(y_predict,0)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_value=roc_auc_score(y_test,y_predict)
print(roc_value)

In [ ]:
df_test.drop(140,axis=1,inplace=True)

In [ ]:
X_target=ss.transform(df_test)

In [ ]:
y_target=clf.predict(df_test)

In [ ]:
y_target1=pd.DataFrame(y_target)
y_target1[y_target1[0]==1].count()

In [ ]:
pd.DataFrame({"UniqueID":df_test_new.index,"loan_default":y_target}).to_csv("submission_without_cat_col.csv")